# SMBO Gaussian Process - New York City Taxi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import seaborn as sns
import pandas as pd

import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score

sns.set()

## Chargement des données

In [ ]:
data = pd.read_csv("nyc_taxi.csv").iloc[:, 1:]
data.head()

In [ ]:
data['pickup_datetime'] = data['pickup_datetime'].str.slice(0, 16)
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
data.head()

## Nettoyage des données

In [ ]:
# Remove observations with missing values
# Since there are only a few of these, i'm not concerned with imputation
data.dropna(how='any', axis='rows', inplace=True)

# Removing observations with erroneous values
mask = data['pickup_longitude'].between(-75, -73)
mask &= data['dropoff_longitude'].between(-75, -73)
mask &= data['pickup_latitude'].between(40, 42)
mask &= data['dropoff_latitude'].between(40, 42)
mask &= data['passenger_count'].between(0, 8)
mask &= data['fare_amount'].between(0, 250)

data = data[mask]

## Feature Engineering

In [ ]:
# TODO : Implémenter la distance de Manhattan (norme L1)
def dist(pickup_lat, pickup_long, dropoff_lat, dropoff_long):  
    distance = # ...
    return distance

In [ ]:
def transform(data):
    # Extract date attributes and then drop the pickup_datetime column
    data['hour'] = data['pickup_datetime'].dt.hour
    data['day'] = data['pickup_datetime'].dt.day
    data['month'] = data['pickup_datetime'].dt.month
    data['year'] = data['pickup_datetime'].dt.year
    data = data.drop('pickup_datetime', axis=1)

    # Distances to nearby airports, and city center
    # By reporting distances to these points, the model can somewhat triangulate other locations of interest
    nyc = (-74.0063889, 40.7141667)
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    data['distance_to_center'] = dist(nyc[1], nyc[0],
                                      data['pickup_latitude'], data['pickup_longitude'])
    data['pickup_distance_to_jfk'] = dist(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = dist(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = dist(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = dist(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = dist(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = dist(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    data['long_dist'] = data['pickup_longitude'] - data['dropoff_longitude']
    data['lat_dist'] = data['pickup_latitude'] - data['dropoff_latitude']
    
    data['dist'] = dist(data['pickup_latitude'], data['pickup_longitude'],
                        data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data


dataset = transform(data)
dataset = dataset.drop(['key'], axis=1)
dataset = dataset.sample(10000)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.drop('fare_amount', axis=1),
                                                    dataset['fare_amount'], test_size=0.25)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [ ]:
dataset.head()

## Optimisation bayésienne des hyper-paramètres

### Minimisation de la fonction de perte

In [ ]:
def xgb_evaluate(max_depth, gamma, colsample_bytree, nrounds):
    params = {'eval_metric': 'rmse',
              'objective':'reg:squarederror',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              'nrounds': int(nrounds)}
    
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3, metrics=['rmse'])    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -cv_result['test-rmse-mean'].iloc[-1]

In [ ]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             'nrounds': (100, 1000)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')

## Entraînement du modèle optimisé

In [ ]:
# TODO : Entraîner un modèle avec le jeu d'hyper-paramètres optimal

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score

print(r2_score(y_test, model.predict(X_test)))